In [1]:
from main import *
import tensorflow as tf

from tqdm import tnrange, tqdm_notebook, tqdm

TensorFlow Version: 1.3.0


/Users/tantony/code/udacity/P12-Semantic-Segmentation/main.py:15: UserWarning: No GPU found. Please use a GPU to train your neural network.
  warnings.warn('No GPU found. Please use a GPU to train your neural network.')


In [2]:
num_classes = 2
image_shape = (160, 576)
epochs = 1
batch_size = 8
data_dir = './data'
runs_dir = './runs'
tests.test_for_kitti_dataset(data_dir)

# Download pretrained vgg model
helper.maybe_download_pretrained_vgg(data_dir)

Tests Passed


In [3]:
sess = tf.Session()

In [4]:
# Path to vgg model
vgg_path = os.path.join(data_dir, 'vgg')

# OPTIONAL: Augment Images for better results
#  https://datascience.stackexchange.com/questions/5224/how-to-prepare-augment-images-for-neural-network

# TODO: Build NN using load_vgg, layers, and optimize function
input_image, keep_prob, layer3_out, layer4_out, layer7_out = load_vgg(sess, vgg_path)
nn_layers = layers(layer3_out, layer4_out, layer7_out, num_classes)
learning_rate = tf.placeholder(tf.float32)
correct_label = tf.placeholder(tf.float32, shape=(None, None, None, num_classes))
logits, train_op, cross_entropy_loss = optimize(nn_layers, correct_label, learning_rate, num_classes)

# Initialize TF variables
sess.run(tf.global_variables_initializer())

INFO:tensorflow:Restoring parameters from b'./data/vgg/variables/variables'


In [5]:
get_batches_fn = helper.gen_batch_function(os.path.join(data_dir, 'data_road/training'), image_shape)

In [7]:
for ep in tnrange(epochs, unit='epochs'):
    for batch_idx, (image, label) in tqdm_notebook(enumerate(get_batches_fn(batch_size)), unit='batches'):
        feed_dict = {input_image: image,
                     correct_label: label,
                     keep_prob: 0.50,
                     learning_rate: 0.0001,
                     }
        _, loss = sess.run([train_op, cross_entropy_loss], feed_dict)
#         if batch_idx % 20 == 0:
#             print('Epoch:',ep,'Batch:',batch_idx,'Loss =',loss)
        tqdm.write('Loss = %.3f'%loss)
        if batch_idx >= 100:
            break


Loss = 52.550
Loss = 35.242
Loss = 29.455
Loss = 23.262
Loss = 19.924
Loss = 17.826
Loss = 14.573
Loss = 12.852



KeyboardInterrupt: 

In [ ]:
helper.save_inference_samples(runs_dir, data_dir, sess, image_shape, logits, keep_prob, input_image)